In [ ]:
%cd /content/drive/MyDrive/ThesisProductClassifier

/content/drive/MyDrive/ThesisProductClassifier


In [ ]:
!pip install tensorflow pillow numpy opencv-python matplotlib SciPy

In [ ]:
!unzip ./dataset/archive.zip -d ./dataset/

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMG_SIZE = [224, 224]
train_folder = 'dataset/train'
test_folder = 'dataset/test'

my_resnet = ResNet50(input_shape= IMG_SIZE + [3], weights = "imagenet", include_top = False)
for layer in my_resnet.layers:
  layer.trainable = False

folders = glob(train_folder + '/*')

plus_flatten_layer = Flatten()(my_resnet.output)
prediction = Dense(len(folders), activation = 'softmax')(plus_flatten_layer)

model = Model(inputs = my_resnet.input, outputs = prediction)
model.summary()

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    train_folder,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)
test_set = test_datagen.flow_from_directory(
    test_folder,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

my_callbacks = [
    ModelCheckpoint(filepath='checkpoint/model.{epoch:02d}-{val_loss:.2f}.h5'),
]

result = model.fit(
    training_set,
    validation_data = test_set,
    epochs = 50,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

In [ ]:
!zip -r dataset.zip dataset

Scanning files .
  adding: dataset/ (stored 0%)
  adding: dataset/companies.csv (deflated 79%)
  adding: dataset/archive.zip (stored 0%)
  adding: dataset/imgs_zip/ (stored 0%)
  adding: dataset/imgs_zip/imgs/ (stored 0%)
  adding: dataset/imgs_zip/imgs/Acura/ (stored 0%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_025.jpg (deflated 7%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_034.jpg (deflated 4%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_040.jpg (deflated 2%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_000.jpg (deflated 3%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_009.jpg (deflated 7%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_032.jpg (deflated 3%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_024.jpg (deflated 3%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_006.jpg (deflated 7%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_014.jpg (deflated 2%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_011.jpg (deflated 3%)
  adding: dataset/imgs_zip/imgs/Acura/Acura_036.jpg (deflated 7%)
 

In [ ]:
plt.plot(result.history['accuracy'], label = 'train_accuracy')
plt.plot(result.history['val_accuracy'], label = 'test_accuracy')
plt.legend()
plt.show()

50


In [ ]:
plt.plot(result.history['loss'], label = 'train_loss')
plt.plot(result.history['val_loss'], label = 'test_loss')
plt.legend()
plt.show()

In [ ]:
model.save('model/car_model.h5')

In [ ]:
"grocery_products": {
    "src": "https://github.com/marcusklasson/GroceryStoreDataset"
},
"medicine_details": {
    "src": "https://www.kaggle.com/datasets/singhnavjot2062001/11000-medicine-details"
}